In [1]:
# Import Necessary Packages

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from imutils.video import VideoStream
import imutils
import numpy as np
import cv2

In [2]:
# Load Face Detector Model

prototxt = r"face_detector/deploy.prototxt"
weights = r"face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxt, weights)

# Load DeepFace Model
DeepFace = load_model("DeepFaceMask.model")

In [3]:
# Detect faces and masks in the given frames

def detect_masks(frame, faceNet, DeepFace):
    (height, width) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (100, 100),
                                 (104.0, 177.0, 123.0))
    
#     Detect Faces In The Given Blob
    faceNet.setInput(blob)
    detections = faceNet.forward()
    #print(detections.shape)
    
#     Initialize Faces, Locations And Predictions
    faces = []
    preds = []
    locs = []
    
#     Looping over Detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
#         Filter out low confidence detections
        if confidence > 0.5:
        
#             Compute BoundingBox for the detection
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype("int")
            
#             Ensure BoundingBox falls in the range of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(width - 1, endX), min(height - 1, endY))
            
#             Extract ROI, Resize and Preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (100, 100))
            face = img_to_array(face)
            face = preprocess_input(face)
            
#             Add Face to The list
            faces.append(face)
            locs.append((startX, startY, endX, endY))
    
    if len(faces) > 0:
        faces = np.array(faces, dtype='float32')
        preds = DeepFace.predict(faces, batch_size=32)
    
#     Return face locations and mask predictions
    return(locs, preds)

In [5]:
# Load VideoStream

stream = VideoStream(src=1).start()

# Loop over the video stream frame by frame
while True:
    frame = stream.read()
    frame = imutils.resize(frame, width=600)
    
    (locs, preds) = detect_masks(frame, faceNet, DeepFace)
    
    for(box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, no_mask) = pred
        
#         Set Label and Box color
        label = 'Mask' if mask > no_mask else 'No Mask'
        color = (0, 255, 0) if label == 'Mask' else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, no_mask) * 100)
        
#         Render Label and BoundingBox on the output frame
        cv2.putText(frame, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2
                   )
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
#     Render the output frame
    cv2.imshow("DeepFace", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break

cv2.destroyAllWindows()
stream.stop()